In [9]:
import torch
from dataset.AmazonFashionDataset import AmazonFashionDataset 
from model.SentimentClassifier import SentimentClassifier
import yaml
from torch.utils.tensorboard import SummaryWriter
import torchtext

In [4]:
def my_collate(batch):
    data, target = zip(*batch)
    # data = torch.tensor(data)
    # print(data)
    # exit()
    data_lens = [len(x) for x in data]
    data_padded = torch.nn.utils.rnn.pad_sequence(data, batch_first=True, padding_value=0)
    print(data_padded)
    print(data_lens)
    #packed = torch.nn.utils.rnn.pack_padded_sequence(data_padded, data_lens, batch_first=True, enforce_sorted=False)
    return data_padded, target, data_lens
    print(packed)
    exit()
    data = [item[0] for item in batch]
    target = [item[1] for item in batch]
    #data = torch.LongTensor(data)

    return batch #[data, target]

In [12]:
yaml_name = "cfg/train_config.yaml"
with open(yaml_name) as f:
    yaml_file = open(yaml_name, 'r')
cfg = yaml.load(yaml_file.read(), Loader=yaml.FullLoader)
writer = SummaryWriter()
dataset = AmazonFashionDataset(cfg)
train_loader = torch.utils.data.DataLoader(dataset, shuffle = False, batch_size=2, collate_fn = my_collate)

model = SentimentClassifier(cfg["MODEL"]["EMBEDDING_SIZE"], cfg["MODEL"]["HIDDEN_SIZE"])
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
temp = 0
for epoch in range(cfg["TRAIN"]["EPOCH"]):
    for i, (data, label, data_lens) in enumerate(train_loader):
        optimizer.zero_grad()
        out = model(data, data_lens)
        #out = out.unsqueeze(0)
        print(out)
        loss = criterion(out, label)
        loss.backward()
        optimizer.step()
        writer.add_scalar("Loss/train", loss, temp )
        temp += 1

tensor([[ 1664,    86,    10,  2379,     1,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0],
        [   10,  2093,    58,    13,   469,  3053,     4,    13,  3650,    32,
           156,  1741,     1,    10,  1145,  4565,    13,  7051,    46,   191,
           520,  6534, 12569,   660,    16,    87,   375,    85,  5574,   285,
            59,    13,   624,    55,   109,    48,   796,    78,   807,     1,
           196,    78,   872,   389,    79,    37,    13,  2858,     4,    78,
          4357,   119,     1]])
[5, 53]
AAAAAA
PackedSequence(data=tensor([[ 0.0653, -0.0464, -0.1589,  ..., -0.1022,  0.1846,  0.0813],
        [ 0.0111, -0.0622, -0.0921,  ..., 

RuntimeError: Index tensor must have the same number of dimensions as input tensor

In [11]:
class SentimentClassifier(torch.nn.Module):
    def __init__(self, embedding_size, hidden_size):
        super(SentimentClassifier, self).__init__()
        self.embedding = torchtext.vocab.GloVe(name='twitter.27B', dim=25)
        self.emdedding_size = embedding_size
        self.hidden_size = hidden_size
        self.lstm = torch.nn.LSTM(input_size = self.emdedding_size, hidden_size = self.hidden_size, batch_first=True)
        self.linear = torch.nn.Linear(self.hidden_size,64)
        self.leaky_relu = torch.nn.LeakyReLU()
        self.linear2 = torch.nn.Linear(64,3)
        self.softmax = torch.nn.Softmax(dim=1)

    def forward(self, x, x_len):
        #x_embed = self.embedding.get_vecs_by_tokens(x)
        print("AAAAAA")

        x_embed = self.embedding.vectors[x]
        x_embed = torch.FloatTensor(x_embed)
        packed = torch.nn.utils.rnn.pack_padded_sequence(x_embed, x_len, batch_first=True, enforce_sorted=False)
        lstm_out, (h_n, c_n) = self.lstm(packed)
        lstm_out_padded, lstm_output_lengths = torch.nn.utils.rnn.pad_packed_sequence(lstm_out, batch_first=True)
        print(lstm_out)
        print(lstm_out_padded[0][4])
        slicer = [i-1 for i in x_len]
        slicer = torch.tensor([[4], [52]])
        print("slicer.shape" , slicer.shape)
        print("before slicing" , lstm_out_padded.shape)
        #lstm_actual_out = lstm_out_padded[:, slicer,:]
        lstm_actual_out = lstm_out_padded.permute(2,0,1)
        lstm_actual_out = torch.gather(lstm_out_padded[:], 0, slicer)
        print("after slicing" , lstm_actual_out.shape)
        exit()
        y1 = self.linear(lstm_out[:, -1])
        y2 = self.leaky_relu(y1)
        y3 = self.linear2(y2)
        y4 = self.softmax(y3)
        return y4

In [16]:
embedding = torchtext.vocab.GloVe(name='twitter.27B', dim=25)
emdedding_size = 25
hidden_size = 64
lstm = torch.nn.LSTM(input_size = 25, hidden_size = 64, batch_first=True)
linear = torch.nn.Linear(64,64)
leaky_relu = torch.nn.LeakyReLU()
linear2 = torch.nn.Linear(64,3)
softmax = torch.nn.Softmax(dim=1)

In [28]:

x_embed = embedding.vectors[data]
x_embed = torch.FloatTensor(x_embed)
packed = torch.nn.utils.rnn.pack_padded_sequence(x_embed, data_lens, batch_first=True, enforce_sorted=False)
lstm_out, (h_n, c_n) = lstm(packed)
lstm_out_padded, lstm_output_lengths = torch.nn.utils.rnn.pad_packed_sequence(lstm_out, batch_first=True)
print(lstm_out)
print(lstm_out_padded[0][4])
slicer = [i-1 for i in data_lens]
slicer = torch.tensor([[4], [52]])
print("slicer.shape" , slicer.shape)
print("before slicing" , lstm_out_padded.shape)
#lstm_actual_out = lstm_out_padded[:, slicer,:]
#lstm_actual_out = lstm_out_padded.permute(2,0,1)
lstm_actual_out = torch.gather(lstm_out_padded, 1, slicer2)
print("after slicing" , lstm_actual_out.shape)

# y1 = self.linear(lstm_out[:, -1])
# y2 = self.leaky_relu(y1)
# y3 = self.linear2(y2)
# y4 = self.softmax(y3)


PackedSequence(data=tensor([[ 0.0701,  0.1503, -0.1439,  ...,  0.0219,  0.0982, -0.0636],
        [ 0.0750,  0.0616, -0.0741,  ..., -0.0333,  0.0171, -0.0309],
        [ 0.0465,  0.1703, -0.2167,  ...,  0.0098, -0.0192, -0.0690],
        ...,
        [ 0.1160,  0.1616, -0.2381,  ...,  0.0278,  0.1881, -0.1064],
        [ 0.0961,  0.2416, -0.2744,  ...,  0.0520,  0.1787, -0.0895],
        [ 0.0526,  0.1784, -0.2569,  ...,  0.1340,  0.1090,  0.0153]],
       grad_fn=<CatBackward0>), batch_sizes=tensor([2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1]), sorted_indices=tensor([1, 0]), unsorted_indices=tensor([1, 0]))
tensor([ 0.0464,  0.1602, -0.2364,  0.1861, -0.0481, -0.0557, -0.1290, -0.0621,
         0.0765, -0.0538, -0.0463,  0.1067, -0.0596,  0.0884,  0.2256,  0.0388,
         0.2230,  0.0056, -0.1956, -0.1015, -0.0595,  0.0375, -0.0452,  0.0195,
        -0.19

In [41]:
lens = torch.tensor([4,52])
lens = lens.unsqueeze(1)
print(lens.shape)
slicer2 = torch.zeros((2,1,64))
slicer2[:] = lens
slicer 
exit()
slicer2[0,0] = 4
slicer2[1,0] = 52
slicer2 = slicer2.type(torch.int64)
slicer2

torch.Size([2, 1])


RuntimeError: The expanded size of the tensor (1) must match the existing size (2) at non-singleton dimension 1.  Target sizes: [2, 1, 64].  Tensor sizes: [2, 1]

In [30]:
lstm_out_padded[0,4,:]

tensor([ 0.0464,  0.1602, -0.2364,  0.1861, -0.0481, -0.0557, -0.1290, -0.0621,
         0.0765, -0.0538, -0.0463,  0.1067, -0.0596,  0.0884,  0.2256,  0.0388,
         0.2230,  0.0056, -0.1956, -0.1015, -0.0595,  0.0375, -0.0452,  0.0195,
        -0.1902,  0.1154, -0.0398,  0.1251, -0.0880, -0.1680, -0.3067,  0.0676,
         0.0506, -0.0269,  0.3212,  0.0867, -0.1190, -0.1213, -0.1276,  0.2491,
        -0.0772, -0.3419,  0.2373,  0.0746, -0.2497,  0.0732, -0.1783, -0.0685,
        -0.1159, -0.1303, -0.0933,  0.1037, -0.0619,  0.1693, -0.0059,  0.1578,
        -0.0660,  0.2270,  0.1718,  0.0863,  0.0490,  0.1306,  0.0997,  0.0325],
       grad_fn=<SliceBackward0>)

In [43]:
lstm_actual_out.squeeze(1).shape

torch.Size([2, 64])

In [36]:
lstm_out_padded.grad

/tmp/ipykernel_8726/2960823341.py:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at aten/src/ATen/core/TensorBody.h:480.)
  lstm_out_padded.grad


In [58]:
slicer  = torch.tensor([7,14,29]).unsqueeze(1)


In [45]:
slicer = x_len

In [63]:
slicer.expand(slicer.shape[0],64).unsqueeze(1) - 1#.shape

tensor([[[ 6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
           6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
           6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
           6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6]],

        [[13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13,
          13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13,
          13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13,
          13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13]],

        [[28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28,
          28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28,
          28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28,
          28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28]]])